## Sagemaker - Sleep Quality

In [1]:
import sklearn # Check Sklearn version
sklearn.__version__

'1.5.2'

## 1. Initialize Boto3 SDK and create S3 bucket. 

In [2]:
import numpy as np
from sagemaker import get_execution_role
import sagemaker
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
import datetime
import time
import tarfile
import boto3
import pandas as pd

sm_boto3 = boto3.client("sagemaker")
sess = sagemaker.Session()
region = sess.boto_session.region_name
bucket = 'fiton-static-files' # Mention the created S3 bucket name here
print("Using bucket " + bucket)

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/rohanchopra/Library/Application Support/sagemaker/config.yaml
Using bucket fiton-static-files


## 3. Data Exploration and Understanding.

In [3]:
df = pd.read_csv("../exercise/sleep_data.csv")

In [4]:
df.head()

,Person ID,Gender,Age,Occupation,Sleep Duration,Quality of Sleep,Physical Activity Level,Stress Level,BMI Category,Blood Pressure,Heart Rate,Daily Steps,Sleep Disorder
0,1,Male,27,Software Engineer,6.1,6,42,6,Overweight,126/83,77,4200,NaN
1,2,Male,28,Doctor,6.2,6,60,8,Normal,125/80,75,10000,NaN
2,3,Male,28,Doctor,6.2,6,60,8,Normal,125/80,75,10000,NaN
3,4,Male,28,Sales Representative,5.9,4,30,8,Obese,140/90,85,3000,Sleep Apnea
4,5,Male,28,Sales Representative,5.9,4,30,8,Obese,140/90,85,3000,Sleep Apnea


In [5]:
df.shape

(374, 13)

In [6]:
df.columns

Index(['Person ID', 'Gender', 'Age', 'Occupation', 'Sleep Duration',
       'Quality of Sleep', 'Physical Activity Level', 'Stress Level',
       'BMI Category', 'Blood Pressure', 'Heart Rate', 'Daily Steps',
       'Sleep Disorder'],
      dtype='object')

In [7]:
# ['Low_Risk','High_Risk'],[0,1]
df = df.drop(['Stress Level', 'Blood Pressure', 'Person ID', 'Occupation'], axis=1)
df = pd.get_dummies(df)

In [8]:
df.columns

Index(['Age', 'Sleep Duration', 'Quality of Sleep', 'Physical Activity Level',
       'Heart Rate', 'Daily Steps', 'Gender_Female', 'Gender_Male',
       'BMI Category_Normal', 'BMI Category_Normal Weight',
       'BMI Category_Obese', 'BMI Category_Overweight',
       'Sleep Disorder_Insomnia', 'Sleep Disorder_Sleep Apnea'],
      dtype='object')

In [9]:
df.shape

(374, 14)

In [10]:
# Find the Percentage of Values are missing
df.isnull().mean() * 100

Age                           0.0
Sleep Duration                0.0
Quality of Sleep              0.0
Physical Activity Level       0.0
Heart Rate                    0.0
Daily Steps                   0.0
Gender_Female                 0.0
Gender_Male                   0.0
BMI Category_Normal           0.0
BMI Category_Normal Weight    0.0
BMI Category_Obese            0.0
BMI Category_Overweight       0.0
Sleep Disorder_Insomnia       0.0
Sleep Disorder_Sleep Apnea    0.0
dtype: float64

In [11]:
features = list(df.columns)
features

['Age',
 'Sleep Duration',
 'Quality of Sleep',
 'Physical Activity Level',
 'Heart Rate',
 'Daily Steps',
 'Gender_Female',
 'Gender_Male',
 'BMI Category_Normal',
 'BMI Category_Normal Weight',
 'BMI Category_Obese',
 'BMI Category_Overweight',
 'Sleep Disorder_Insomnia',
 'Sleep Disorder_Sleep Apnea']

In [12]:
label = 'Quality of Sleep'
features.remove(label)
label

'Quality of Sleep'

In [13]:
x = df[features]
y = df[label]

In [14]:
x.head()

,Age,Sleep Duration,Physical Activity Level,Heart Rate,Daily Steps,Gender_Female,Gender_Male,BMI Category_Normal,BMI Category_Normal Weight,BMI Category_Obese,BMI Category_Overweight,Sleep Disorder_Insomnia,Sleep Disorder_Sleep Apnea
0,27,6.1,42,77,4200,False,True,False,False,False,True,False,False
1,28,6.2,60,75,10000,False,True,True,False,False,False,False,False
2,28,6.2,60,75,10000,False,True,True,False,False,False,False,False
3,28,5.9,30,85,3000,False,True,False,False,True,False,False,True
4,28,5.9,30,85,3000,False,True,False,False,True,False,False,True


In [15]:
# {0: 'Low_Risk',1: 'High_Risk'}
y.head()

0    6
1    6
2    6
3    4
4    4
Name: Quality of Sleep, dtype: int64

In [16]:
x.shape

(374, 13)

In [17]:
y.value_counts()

Quality of Sleep
8    109
6    105
7     77
9     71
5      7
4      5
Name: count, dtype: int64

In [18]:
X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=0.25, random_state=42, shuffle = True)

In [19]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(280, 13)
(94, 13)
(280,)
(94,)


## 4. Split the data into Train/Test CSV File. 

In [20]:
trainX = pd.DataFrame(X_train)
trainX[label] = y_train

testX = pd.DataFrame(X_test)
testX[label] = y_test

In [21]:
print(trainX.shape)
print(testX.shape)

(280, 14)
(94, 14)


In [22]:
trainX.head()

,Age,Sleep Duration,Physical Activity Level,Heart Rate,Daily Steps,Gender_Female,Gender_Male,BMI Category_Normal,BMI Category_Normal Weight,BMI Category_Obese,BMI Category_Overweight,Sleep Disorder_Insomnia,Sleep Disorder_Sleep Apnea,Quality of Sleep
222,44,6.3,45,72,6000,False,True,False,False,False,True,True,False,6
227,44,6.3,45,72,6000,False,True,False,False,False,True,True,False,6
141,38,7.1,60,68,8000,False,True,True,False,False,False,False,False,8
17,29,6.0,30,70,8000,False,True,True,False,False,False,False,True,6
246,44,6.3,45,72,6000,False,True,False,False,False,True,True,False,6


In [23]:
trainX.isnull().sum()

Age                           0
Sleep Duration                0
Physical Activity Level       0
Heart Rate                    0
Daily Steps                   0
Gender_Female                 0
Gender_Male                   0
BMI Category_Normal           0
BMI Category_Normal Weight    0
BMI Category_Obese            0
BMI Category_Overweight       0
Sleep Disorder_Insomnia       0
Sleep Disorder_Sleep Apnea    0
Quality of Sleep              0
dtype: int64

In [24]:
testX.isnull().sum()

Age                           0
Sleep Duration                0
Physical Activity Level       0
Heart Rate                    0
Daily Steps                   0
Gender_Female                 0
Gender_Male                   0
BMI Category_Normal           0
BMI Category_Normal Weight    0
BMI Category_Obese            0
BMI Category_Overweight       0
Sleep Disorder_Insomnia       0
Sleep Disorder_Sleep Apnea    0
Quality of Sleep              0
dtype: int64

## 5. Upload data into the S3 Bucket.

In [25]:
trainX.to_csv("train-V-1.csv",index = False)
testX.to_csv("test-V-1.csv", index = False)

In [26]:
# send data to S3. SageMaker will take training data from s3
sk_prefix = "sagemaker/health-data/sklearncontainer"
trainpath = sess.upload_data(
    path="train-V-1.csv", bucket=bucket, key_prefix=sk_prefix
)

testpath = sess.upload_data(
    path="test-V-1.csv", bucket=bucket, key_prefix=sk_prefix
)

In [27]:
testpath

's3://fiton-static-files/sagemaker/health-data/sklearncontainer/test-V-1.csv'

In [28]:
trainpath

's3://fiton-static-files/sagemaker/health-data/sklearncontainer/train-V-1.csv'

In [ ]:
sleep_models_dir = 's3://ml-data-repository-pp2959/sagemaker/sleep-models'

## 6. Create Training Script

In [29]:
%%writefile sleep_script.py


from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import validation_curve, train_test_split

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_score, recall_score, f1_score, roc_curve, auc
import sklearn
import joblib
import boto3
import pathlib
from io import StringIO 
import argparse
import joblib
import os
import numpy as np
import pandas as pd
import boto3

s3 = boto3.client("s3")


# inference functions ---------------

def input_fn(request_body, request_content_type):
    print(request_body)
    print(request_content_type)
    if request_content_type == "text/csv":
        request_body = request_body.strip()
        try:
            df = pd.read_csv(StringIO(request_body), header=None)
            return df
        
        except Exception as e:
            print(e)
    else:
        return """Please use Content-Type = 'text/csv' and, send the request!!""" 
 
    
def model_fn(model_dir):
    clf = joblib.load(os.path.join(model_dir, "model.joblib"))
    return clf

def predict_fn(input_data, model):
    if type(input_data) != str:
        prediction = model.predict(input_data)
        print(prediction)
        return prediction
    else:
        return input_data
        
    
if __name__ == "__main__":

    print("[INFO] Extracting arguments")
    parser = argparse.ArgumentParser()

    # hyperparameters sent by the client are passed as command-line arguments to the script.
    parser.add_argument("--n_estimators", type=int, default=100)
    parser.add_argument("--random_state", type=int, default=0)

    # Data, model, and output directories
    parser.add_argument("--model-dir", type=str, default=os.environ.get("SM_MODEL_DIR"))
    parser.add_argument("--train", type=str, default=os.environ.get("SM_CHANNEL_TRAIN"))
    parser.add_argument("--test", type=str, default=os.environ.get("SM_CHANNEL_TEST"))
    parser.add_argument("--train-file", type=str, default="train-V-1.csv")
    parser.add_argument("--test-file", type=str, default="test-V-1.csv")
    parser.add_argument("--s3-bucket", type=str, default="fiton-static-files")
    parser.add_argument("--s3-data-key", type=str, default="sagemaker/health-data/sklearncontainer/")
    parser.add_argument("--s3-model-key", type=str, default="sagemaker/sleep-models/")

    args, _ = parser.parse_known_args()
    
    print("SKLearn Version: ", sklearn.__version__)
    print("Joblib Version: ", joblib.__version__)

    print("[INFO] Reading data")
    print()
    
    train_path = os.path.join(args.train, args.train_file)
    test_path = os.path.join(args.test, args.test_file)
    s3.download_file(args.s3_bucket, args.s3_data_key + args.train_file, train_path)
    s3.download_file(args.s3_bucket, args.s3_data_key + args.test_file, test_path)
    
    train_df = pd.read_csv(train_path)
    test_df = pd.read_csv(test_path)
    
    print(train_df.head())
    print(test_df.head())
    
    features = list(train_df.columns)
    label = features.pop(-1)
    
    print("Building training and testing datasets")
    print()
    X_train = train_df[features]
    X_test = test_df[features]
    y_train = train_df[label]
    y_test = test_df[label]

    print('Column order: ')
    print(features)
    print()
    
    print("Label column is: ",label)
    print()
    
    print("Data Shape: ")
    print()
    print("---- SHAPE OF TRAINING DATA (85%) ----")
    print(X_train.shape)
    print(y_train.shape)
    print()
    print("---- SHAPE OF TESTING DATA (15%) ----")
    print(X_test.shape)
    print(y_test.shape)
    print()
    
#     print("Determining best value of K using K Fold CV ......")
#     print()
    
#     max_k = 10
#     n_fold = 10
#     train_scores, valid_scores = validation_curve(
#                             KNeighborsClassifier(),
#                             X_train, y_train,
#                             param_name = "n_neighbors",
#                             param_range = np.arange(1, max_k, 2),
#                             cv = n_fold)
    
#     acc_val_mean = valid_scores.mean(axis=1)
#     acc_val_se   = valid_scores.std(axis=1)/np.sqrt(n_fold-1)

#     acc_tr_mean = train_scores.mean(axis=1)
#     acc_tr_se   = train_scores.std(axis=1)/np.sqrt(n_fold-1)
    
    # get largest number of neighbors (simplest model)
    # where mean score is still greater than target
#     idx_1se = max(np.where(acc_val_mean >= target)[0])
#     nn_1se = nn_list[idx_1se]
#     print("Best model has number of neighbors: %d" % nn_1se)
    
    print("Training KnearestNeighbour Model.....")
    print()
    
#     model = KNeighborsRegressor(n_neighbors = nn_1se)
#     model.fit(X_train, y_train)
    
    model = KNeighborsClassifier(n_neighbors = 3)
    model.fit(X_train, y_train)
    print()
    

    model_path = os.path.join(args.model_dir, "model.joblib")
    joblib.dump(model,model_path)
    s3.upload_file(model_path, args.s3_bucket, args.s3_model_key + "model.joblib")
    print("Model persisted at " + model_path)
    print()

    
    y_pred_test = model.predict(X_test)
    
    
    test_acc = accuracy_score(y_test,y_pred_test)
    test_rep = classification_report(y_test,y_pred_test)

    print()
    print("---- METRICS RESULTS FOR TESTING DATA ----")
    print()
    print("Total Rows are: ", X_test.shape[0])
    print('[TESTING] Model Accuracy is: ', test_acc)
    print('[TESTING] Testing Report: ')
    print(test_rep)

Writing sleep_script.py


In [30]:
! python sleep_script.py --n_estimators 100 \
                   --random_state 0 \
                   --model-dir ./ \
                   --train ./ \
                   --test ./ \

[INFO] Extracting arguments
SKLearn Version:  1.5.2
Joblib Version:  1.4.2
[INFO] Reading data

   Age  Sleep Duration  ...  Sleep Disorder_Sleep Apnea  Quality of Sleep
0   44             6.3  ...                       False                 6
1   44             6.3  ...                       False                 6
2   38             7.1  ...                       False                 8
3   29             6.0  ...                        True                 6
4   44             6.3  ...                       False                 6

[5 rows x 14 columns]
   Age  Sleep Duration  ...  Sleep Disorder_Sleep Apnea  Quality of Sleep
0   53             8.5  ...                       False                 9
1   31             6.1  ...                       False                 6
2   29             6.0  ...                       False                 6
3   53             8.5  ...                       False                 9
4   32             6.0  ...                       False            

## 7. Train script inside Sagemaker container.

In [32]:
from sagemaker.sklearn.estimator import SKLearn

FRAMEWORK_VERSION = "0.23-1"

sklearn_estimator = SKLearn(
    entry_point="sleep_script.py",
    role="arn:aws:iam::225989339915:role/aws-elasticbeanstalk-service-role",
    instance_count=1,
    instance_type="ml.m5.large",
    framework_version=FRAMEWORK_VERSION,
    base_job_name="RF-custom-sklearn",
    hyperparameters={
        "n_estimators": 100,
        "random_state": 0,
    },
    use_spot_instances = True,
    max_wait = 7200,
    max_run = 3600
)

In [39]:
# launch training job, with asynchronous call
sklearn_estimator.fit({"train": trainpath, "test": testpath}, wait=True)
# sklearn_estimator.fit({"train": datapath}, wait=True)

INFO:sagemaker:Creating training-job with name: RF-custom-sklearn-2024-11-27-17-56-08-160


2024-11-27 17:56:09 Starting - Starting the training job...
2024-11-27 17:56:22 Starting - Preparing the instances for training...
2024-11-27 17:57:15 Downloading - Downloading the training image.....2024-11-27 17:58:01,402 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2024-11-27 17:58:01,415 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-11-27 17:58:01,499 sagemaker_sklearn_container.training INFO     Invoking user training script.
2024-11-27 17:58:01,758 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-11-27 17:58:01,782 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-11-27 17:58:01,807 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-11-27 17:58:01,823 sagemaker-training-toolkit INFO     Invoking user script
Training Env:
{
    "additional_framework_parameters": {},
    "channel_input_di

## 8. Store Model Artifacts(model.tar.gz) into the S3 Bucket. 

In [40]:
sklearn_estimator.latest_training_job.wait(logs="None")
artifact = sm_boto3.describe_training_job(
    TrainingJobName=sklearn_estimator.latest_training_job.name
)["ModelArtifacts"]["S3ModelArtifacts"]

print("Model artifact persisted at " + artifact)


2024-11-27 17:58:20 Starting - Preparing the instances for training
2024-11-27 17:58:20 Downloading - Downloading the training image
2024-11-27 17:58:20 Training - Training image download completed. Training in progress.
2024-11-27 17:58:20 Uploading - Uploading generated training model
2024-11-27 17:58:20 Completed - Training job completed
Model artifact persisted at s3://sagemaker-us-west-2-225989339915/RF-custom-sklearn-2024-11-27-17-56-08-160/output/model.tar.gz


## 9. Deploy Sagemaker Endpoint(API) for trained model, and test it. 

In [48]:
from sagemaker.sklearn.model import SKLearnModel
from time import gmtime, strftime

model_name = "Custom-sklearn-model-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print(artifact)
model = SKLearnModel(
    name =  model_name,
    model_data=artifact,
    role="arn:aws:iam::225989339915:role/aws-elasticbeanstalk-service-role",
    entry_point="sleep_script.py",
    framework_version=FRAMEWORK_VERSION,
)
print(model)

s3://sagemaker-us-west-2-225989339915/RF-custom-sklearn-2024-11-27-17-56-08-160/output/model.tar.gz


In [56]:
import boto3

# Initialize SageMaker client
sagemaker_client = boto3.client('sagemaker')

# Specify the S3 location of the new model artifact
new_model_url = artifact

# Specify the name of the existing SageMaker endpoint configuration
endpoint_config_name = 'Sleep-Quality-Inference-Model-1'

# Specify the name of the existing SageMaker endpoint
endpoint_name = 'Sleep-Quality-Inference-Model'

try:
    response = sagemaker_client.create_endpoint_config(
        EndpointConfigName = endpoint_config_name,
        ProductionVariants=[{
            'VariantName': 'variant-1',
            'ModelName': model_name,
            'InitialInstanceCount': 1,
            'InstanceType': 'ml.m5.large',
            'InitialVariantWeight': 1
        }]
    )
    print(response)
except Exception as e:
    print(f"Endpoint already exists... {e}")



{'EndpointConfigArn': 'arn:aws:sagemaker:us-west-2:225989339915:endpoint-config/Sleep-Quality-Inference-Model-1', 'ResponseMetadata': {'RequestId': '2e1c6e1d-0cdc-4826-89bb-e63b841ecf89', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '2e1c6e1d-0cdc-4826-89bb-e63b841ecf89', 'content-type': 'application/x-amz-json-1.1', 'content-length': '112', 'date': 'Wed, 27 Nov 2024 18:33:57 GMT'}, 'RetryAttempts': 0}}


In [57]:
# Update the SageMaker endpoint configuration to use the updated model
response = sagemaker_client.update_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name,
    RetainAllVariantProperties=True,
)

# Print the response
print(response)


{'EndpointArn': 'arn:aws:sagemaker:us-west-2:225989339915:endpoint/Sleep-Quality-Inference-Model', 'ResponseMetadata': {'RequestId': 'b1f7254a-1a87-4c87-b091-35a31bc78a7a', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'b1f7254a-1a87-4c87-b091-35a31bc78a7a', 'content-type': 'application/x-amz-json-1.1', 'content-length': '97', 'date': 'Wed, 27 Nov 2024 18:34:02 GMT'}, 'RetryAttempts': 0}}


In [49]:
endpoint_name = "Sleep-Quality-Inference-Model"
print("EndpointName={}".format(endpoint_name))

try:
    predictor = model.deploy(
        initial_instance_count=1,
        instance_type="ml.m4.xlarge",
        endpoint_name=endpoint_name,
        update_endpoint = True
    )
except Exception as e:
    print(f"Endpoint already exists... {e}")



See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


EndpointName=Sleep-Quality-Inference-Model


INFO:sagemaker:Creating model with name: Custom-sklearn-model-2024-11-27-18-11-21
INFO:sagemaker:Creating endpoint-config with name Sleep-Quality-Inference-Model
INFO:sagemaker:Creating endpoint with name Sleep-Quality-Inference-Model


-----!

In [58]:
testX.head()

,Age,Sleep Duration,Physical Activity Level,Heart Rate,Daily Steps,Gender_Female,Gender_Male,BMI Category_Normal,BMI Category_Normal Weight,BMI Category_Obese,BMI Category_Overweight,Sleep Disorder_Insomnia,Sleep Disorder_Sleep Apnea,Quality of Sleep
329,53,8.5,30,65,5000,True,False,True,False,False,False,False,False,9
33,31,6.1,30,72,5000,False,True,True,False,False,False,False,False,6
15,29,6.0,30,70,8000,False,True,True,False,False,False,False,False,6
325,53,8.5,30,65,5000,True,False,True,False,False,False,False,False,9
57,32,6.0,30,72,5000,False,True,True,False,False,False,False,False,6


In [59]:
test_features = np.array(testX[features][0:2].values.tolist()[:-1])
test_features

array([[5.3e+01, 8.5e+00, 3.0e+01, 6.5e+01, 5.0e+03, 1.0e+00, 0.0e+00,
        1.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00]])

In [60]:
predictor.predict([[0]*13])

array("Please use Content-Type = 'text/csv' and, send the request!!",
      dtype='<U60')

In [61]:
print(predictor.predict(testX[features][0:2].values.tolist()))

Please use Content-Type = 'text/csv' and, send the request!!


## Don't forget to delete the endpoint !

In [62]:
sm_boto3.delete_endpoint(EndpointName=endpoint_name)

ClientError: An error occurred (ValidationException) when calling the DeleteEndpoint operation: Cannot update in-progress endpoint "arn:aws:sagemaker:us-west-2:225989339915:endpoint/Sleep-Quality-Inference-Model".

### Don't forget to Subscribe Machine Learning Hub YouTube Channel. 